# 04. News classification with LLM and RAG

Author: [ArtyomR](https://github.com/ArtyomR)  
Source: 

<span style="color:red">This Notebook was tested in Google Colab with GPU.</span>.

**In this Notebook I'll walk through News classification with LLM and RAG**  
Next steps were implemented.
1. Download and intilalize Tokinizer and LLM
2. Create function for embedings
3. Create prompt
4. Create LLM answering function
5. Upload and prepare dataset
6. News classification
7. Prediction scorring

Dataset was creted within [Chinese AI news agregation project](https://t.me/chinese_ai_news).  

## Importing Necessary Libraries

In [1]:
!pip -q install peft transformers langchain sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate
import tqdm

In [ ]:
## Download and intialize models

In [3]:
%%time
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
device_map = {"": 0}

model = AutoPeftModelForCausalLM.from_pretrained(
    adapt_model_name,
    device_map=device_map,
    torch_dtype=torch.bfloat16)

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

CPU times: user 23.6 s, sys: 39.6 s, total: 1min 3s
Wall time: 2min 59s


In [4]:
# Load model from HuggingFace Hub
sent_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
sent_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [5]:
def get_embedding(sentence):

    #Mean Pooling - Take attention mask into account for correct averaging
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Tokenize sentences
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = sent_model(**encoded_input)

    # Perform pooling
    sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

In [6]:
prompt_china_ai = PromptTemplate.from_template("user: {news_info}. Содержит ли этот текст информацию о китайском искуственном интеллекте?\nbot: Вот ответ на ваш вопрос длиной в одно слово (Да или Нет):")

In [7]:
def get_answer(info_prompt, news_info):

    prompt = info_prompt.format(news_info=news_info)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                            top_p=0.5,
                            temperature=0.3,
                            attention_mask=inputs["attention_mask"],
                            max_new_tokens=100,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    parsed_answer = output.split("Вот ответ на ваш вопрос длиной в одно слово (Да или Нет):")[1].strip()

    if "bot:" in parsed_answer:
        parsed_answer = parsed_answer.split("bot:")[0].strip()

    # parsed_answer = output.split("bot:")[1].strip()
    return parsed_answer

### Test the model with one news.
Question in the prompt is "Содержит ли этот текст информацию о китайском искуственном интеллекте?"

In [8]:
news_info="Tencent Cloud запускает продукт для рисования с использованием искусственного интеллекта, поддерживающий более 25 созданных стилей 10 сентября IT House сообщила, что на конференции Tencent Global Digital Ecology Conference 2023, состоявшейся 7 сентября, компания Tencent Cloud официально представила новый продукт для рисования с использованием искусственного интеллекта, который использует собственную модель рисования с использованием искусственного интеллекта Tencent для обеспечения технологии создания и редактирования изображений с использованием искусственного интеллекта. позволяет пользователям легко создавать связанный с вводом графический контент на основе введенного текста или изображений. Согласно официальному сообщению, продукт для рисования с использованием искусственного интеллекта, выпущенный Tencent Cloud, на этот раз основан на модели алгоритма рисования, разработанной самостоятельно, и специально оптимизирован для возможностей создания рисунков в китайском контексте. Продукт включает в себя такие функции, как интеллектуальный Wenshengtu и интеллектуальный Wenshengtu, предоставляющий пользователям высокоточное создание изображений и преобразование стилей, которые поддерживают 25 различных стилей сцены, таких как китайский стиль, анимация, игры и традиционные картины."

Correct answer is Да (Yes).

In [9]:
%%time
answer = get_answer(prompt_china_ai, news_info)
print(f'Answer from model: {answer}')

Answer from model: Да.
CPU times: user 10.9 s, sys: 905 ms, total: 11.8 s
Wall time: 13.2 s


### Upload test dataset

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
test_data  = pd.read_excel('/content/drive/MyDrive/Data2/ai_articles_test_231124.xlsx',index_col=0)
test_data.head(2)

,published_flag,combined_text_ru,combined_text_ru_clean
1167,1,Tencent Cloud запускает продукт для рисования ...,tencent cloud запускать продукт для рисование ...
1542,1,Генеративный ИИ достиг «пика завышенных ожидан...,генеративный ия достигнуть пик завысить ожидан...


In [12]:
len(test_data)

926

### Split dataset for features (texts) and classes (labels)

In [13]:
texts = list(test_data['combined_text_ru'][:5])
labels = list(test_data['published_flag'][:5])

### Classification prediction

In [ ]:
%%time
predictions = [get_answer(prompt_china_ai, t) for t in texts]

CPU times: user 43.9 s, sys: 3.27 s, total: 47.2 s
Wall time: 48 s


In [ ]:
predictions[:5]

['Да.', 'Нет.', 'Нет.', 'Да.', 'Нет.']

In [14]:
texts = list(test_data['combined_text_ru'])
labels = list(test_data['published_flag'])

In [15]:
from tqdm import tqdm

In [16]:
%%time
# List to store the answers
predictions = []

# Using tqdm to create a progress bar
for t in tqdm(texts, desc="Processing texts", unit="text"):
    answer = get_answer(prompt_china_ai, t)
    predictions.append(answer)

Processing texts: 100%|██████████| 926/926 [2:16:34<00:00,  8.85s/text]

CPU times: user 2h 7min 48s, sys: 7min 53s, total: 2h 15min 42s
Wall time: 2h 16min 34s


In [17]:
len(predictions)

926

In [21]:
predictions[:10]

['Да.', 'Нет.', 'Нет.', 'Да.', 'Да.', 'Да.', 'Да.', 'Нет.', 'Да.', 'Да.']

In [22]:
dig_predictions = [1 if item == 'Да.' else 0 for item in predictions]
dig_predictions[:10]

[1, 0, 0, 1, 1, 1, 1, 0, 1, 1]

In [23]:
from sklearn.metrics import accuracy_score, classification_report

In [24]:
# Model accuracy
accuracy = accuracy_score(labels, dig_predictions)
print(f'Accuracy: {accuracy}')

# Additional metrics
print(classification_report(labels, dig_predictions))

Accuracy: 0.7224622030237581
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       525
           1       0.67      0.72      0.69       401

    accuracy                           0.72       926
   macro avg       0.72      0.72      0.72       926
weighted avg       0.73      0.72      0.72       926



In [26]:
from sklearn.metrics import f1_score

In [27]:
f1_score(labels, dig_predictions, average='macro')

0.7195100247392012

In [28]:
f1_score(labels, dig_predictions, average='micro')

0.7224622030237581

In [29]:
f1_score(labels, dig_predictions, average='weighted')

0.7233633942895703

## Reference
- [Русский LLM-помощник (saiga) с кэшем, используя RAG (Retrieval-Augmented Generation)](https://habr.com/ru/articles/769124/)